# Data analysis

Endogenous Rhythmic Attention project, 2019-2022

<b>Author</b>: Olof J. van der Werf
<br><b>Last updated</b>: 10-08-2022

[reference + DOI to publication]

### Purpose of this notebook

This notebook analyses the data using single-trial least squares spectrum analysis (stLSS)

<ul>
<li>...</li>
</ul>

### Import necessary libraries

In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Functions

In [402]:
# element-wise convolution with a gaussian in the time domain
def convolution(data):
    
    # element-wise subtraction to put in gaussian function
    arr = []
    for t in data['time']:
        arr.append(t - intervals)
        
    # gaussian function
    W = np.exp(-(np.power(arr,2))/(2*sigma**2))

    # element-wise multiplication of gussian with values
    H = []
    for i,interval in enumerate(intervals):
        H.append(W[:,i] * data['rt'])
    H = np.transpose(H)
    
    # take sum of each to get time series
    W_sum = np.sum(W,0)
    H_sum = np.sum(H,0)
    ts = H_sum / W_sum
    
    return ts

In [403]:
# element-wise detrending of the data 
def detrend(data,ts):
    coeff = np.polyfit(intervals,ts,1)
    trend = np.polyval(coeff,intervals)
    trend = pd.Series(trend,index = intervals)

    detrended = data.copy()
    detrended['rt'] = ''

    for i, row in data.iterrows():
        detrended.loc[i,'rt'] = data.loc[i,'rt'] - trend[detrended.loc[i,'time']] 

    return detrended


In [404]:
# single trial least squares spectrum analysis
def stLSS(data):
    
    # hanning taper
    taper = pd.Series(np.hanning(len(intervals)),index = intervals)
    taper = taper / np.sum(taper)
    data = data.reset_index(drop=True)
    for i, row in data.iterrows():
        data.loc[i,'rt'] = data.loc[i,'rt'] * taper[data.loc[i,'time']] 

    # stLSS
    power_spectrum = pd.Series(index=frequencies,dtype='float64')
    for frequency in frequencies:
        phase = data['time'] * 2 * np.pi * frequency
        phase = phase.to_numpy()
        x = [np.ones(len(data['time'])),np.cos(phase),np.sin(phase)]
        x = np.transpose(x)
        y = np.transpose([data['rt'].to_numpy().astype('float64')])
        x,resid,rank,s = np.linalg.lstsq(x,y,rcond=None)
        power_spectrum[frequency] = complex(x[1][0],x[2][0])
        powerspec = abs(powerspec)**2
        
    return power_spectrum

### Set variables

In [2]:
### Set variables# main variables
clean_data_folder = '/Volumes/fpn_rdm$/DM0874_OW_EndoRhythSamp/09_Data_after_cleaning/';

# time bins
start_time = 0.491
end_time = 1.690
resolution = 0.001
num_time_bins = int((end_time - start_time) / resolution) + 2
intervals = np.round(np.linspace(start_time, end_time, num = num_time_bins),3)

# frequency bins
low_freq = 2
high_freq = 20
resolution = 0.1
num_freq_bins = int((high_freq - low_freq) / resolution) + 1
frequencies =  np.linspace(low_freq, high_freq, num = num_freq_bins)

# sigma of the Gaussian for the convolution
sigma = 0.01

# number of permutations
nr_of_permutations = 1000

# loop lists
conditions = ['60','80','100']
validities = ['valid','invalid']
visual_fields = ['left','right','total']
subjects = ['03','04','05','06','09','11','12','14','15','17','18','19','20','21','26','27','30','31','32','33','34','35','37','38','39','40']

### Import data

In [420]:
# Main loop
data = {}
for validity in validities:
    data[validity] = {}
    
    for visual_field in visual_fields:
        data[validity][visual_field] = pd.DataFrame()
        
        for condition in conditions:
            
            if validity == 'invalid' and condition == '100':
                continue
            
            # subject loop
            for subject in subjects:
                file = clean_data_folder + 'trials/' + condition + '_' + validity + '_' + visual_field + '_' + subject + '.csv'
                subject_data = pd.read_csv(file, sep = ',', header=None, names = ['time','rt'])
                subject_data['interval'] = np.round(subject_data['time'],3)
                subject_data['condition'] = condition
                subject_data['subject'] = subject
                
                # convolute
                ts = convolution(subject_data)
                
                # detrend
                subject_data = detrend(subject_data,ts)
                
                # add to data
                data[validity][visual_field][condition] = pd.concat((data[validity][visual_field],subject_data))
            
                # 
                data[validity][visual_field] = data[validity][visual_field].reset_index(drop=True).astype('float64')
            
            d
            
        

In [424]:
data[validity][visual_field].reset_index(drop=True).astype('float64')

time          rt  condition  subject
0     1.667  224.169577       60.0      3.0
1     0.700  160.521761       60.0      3.0
2     0.567  174.048930       60.0      3.0
3     1.550  272.652079       60.0      3.0
4     1.017   13.250922       60.0      3.0
...     ...         ...        ...      ...
5899  1.583 -145.750205       80.0     40.0
5900  1.567  -48.047096       80.0     40.0
5901  1.650 -108.335474       80.0     40.0
5902  0.633   88.505892       80.0     40.0
5903  0.533  292.807324       80.0     40.0

[5904 rows x 4 columns]

In [418]:
# stLSS analysis
for validity in validities:
    for visual_field in visual_fields:
        for condition in conditions:
            
            
            
            powerspec = stLSS(data)
            
            
            # time bin permutations
            time_permutations = pd.DataFrame()
            
            for i in range(nr_of_permutations):
                perm = data.copy()
                perm['time'] = np.random.permutation(perm['time'])
            
            # convolute, detrend, stLSS
            ts_perm = convolution(perm)
            perm = detrend()
            
            
        
        # condition permutations

     time          rt  condition
0   1.383  -84.942343         60
1   1.183   151.12452         60
2   1.367  -27.156234         60
3   0.750  -47.597592         60
4   0.550  -121.83073         60
..    ...         ...        ...
62  1.583 -145.750205         60
63  1.567  -48.047096         60
64  1.650 -108.335474         60
65  0.633   88.505892         60
66  0.533  292.807324         60

[67 rows x 3 columns]
     time          rt  condition
0   1.150  -84.942343         60
1   1.400   151.12452         60
2   0.800  -27.156234         60
3   1.567  -47.597592         60
4   0.617  -121.83073         60
..    ...         ...        ...
62  0.767 -145.750205         60
63  0.833  -48.047096         60
64  0.917 -108.335474         60
65  1.333   88.505892         60
66  0.983  292.807324         60

[67 rows x 3 columns]
     time          rt  condition
0   1.383  -84.942343         60
1   1.183   151.12452         60
2   1.367  -27.156234         60
3   0.750  -47.597592         